In [18]:
from sage.all import *
import sys
import hashlib
import random
import math
from itertools import chain

In [2]:
key_id = '0'
data_path = './data/'
key_suffix = '80' + '_' + key_id
# gmsk = load(data_path + 'sk' + key_suffix)
pk = load(data_path + 'pk')

In [3]:
show(pk)

(352 x 170 dense matrix over Finite Field of size 2,
 352 x 2048 dense matrix over Finite Field of size 2,
 352 x 2048 dense matrix over Finite Field of size 2,
 1696 x 2048 dense matrix over Finite Field of size 2)


In [4]:
lib_path = './lib/'
load(lib_path + 'common.sage')
load(lib_path + 'goppacode.sage')
load(lib_path + 'JoinImpl.sage')
# load(lib_path + 'KeyPrepare.sage')

In [ ]:
# gpk,gsk = Join(U,B,A,S_A,P_A,H_Goppa,g1,X,SyndromeCalculator,L,2048,170,0.01,1)

In [5]:
key_id = '0'
data_path = './data/'
key_suffix = '80' + '_' + key_id


gpk = load(data_path + 'gpk'+ key_suffix)
gsk = load(data_path + 'gsk'+ key_suffix)

In [8]:
import math

def my_log2(p):
    return math.log(p)/math.log(2)

def log_comb(n, k):
    if not hasattr(log_comb, 'dict'):
        log_comb.dict = dict()
    my_dict = log_comb.dict
    vlu = my_dict.get((n,k), -1)
    if vlu < 0:
        my_dict[(n,k)]=vlu=sum([my_log2(n-j)-my_log2(j+1) for j in range(k)])
    return vlu

def compute_truncate_pr(n, p, lb, ub):
    '''
    Compute sum_{lb<=j<=ub}binom{n}{j}p^j (1-p)^{n-j}
    '''
    return sum([2**(log_comb(n, j)+j*my_log2(p)+(n-j)*my_log2(1-p)) for j in range(lb, ub+1)])

def compute_rejection_sampling_pr(n, p, s, t):
    '''
    Compute the ratio of binomial distribution over shifted binomial distribution.
    '''
    log_comb_n_s = log_comb(n, s)
    return 1/sum([2**(log_comb(t, j)+log_comb(n-t, s-j)-log_comb_n_s+(s-2*j)*my_log2(p)+(2*j-s)*my_log2(1-p)) for j in range(s+1)])


def rejection_sampling_rate(N, s, tau, Xi):
    rate_dict = dict()

    lb = int(N*tau)-Xi; ub = int(N*tau)+Xi

    # denominator of truncated binomial distribution density function
    dem_trun_pr = compute_truncate_pr(N, tau, lb, ub)
    for t in range(lb, ub+1):
        # compute the rejection rate for t
        rejection_sampling_pr = compute_rejection_sampling_pr(N,tau,s,t)/dem_trun_pr
        rate_dict[t] = rejection_sampling_pr
    M = max(rate_dict.values())
    return rate_dict, M

In [9]:
def hash_function_1(n, y,msg,field):
    r = int(hashlib.sha512((str(y)+msg).encode('utf-8')).hexdigest(), base=16)
#     s = set()
    lst = n*[0]
    lst = vector(field,lst)
    while r != 0:
        ele = r%n
        r = r//n
        lst[ele] = lst[ele] + 1
    return lst


In [10]:
def hash_function_2(n, w, t, y, msg, field):
    r = int(hashlib.sha512((str(t)+str(y)+msg).encode('utf-8')).hexdigest(), base=16)
    s = set()
    while len(s) < w and r != 0:
        s.add(r%n)
        r = r//n
    if len(s) >= w:
        lst = n*[0]
        for ele in s:
            lst[ele] = 1
        lst = vector(field,lst)
        return lst
    else:
        print ('Error in hash function')


In [20]:
def sign( msg,n,k,l, w,sigma_1,sigma_2, tau,xi,pk,gsk,field):
#     random.seed(msg)
    X_i1 = gsk[0]
    X_i2 = gsk[1]
#     U = pk[0]
    B = pk[1]
    A = pk[2]
    G_pub = pk[3]
    
#     R=PolynomialRing(GF(2), 'x', implementation='NTL');x=R.gen();S=R.quotient(x^n-1, 'a');a = S.gen()
#     h = S(pk)
#     s1_lst, s2_lst = sk
#     s1 = S(s1_lst); s2 = S(s2_lst)
    counter = 0
    eta_1 = 0.5 * (1-(1-2*sigma_1)**w)
    eta_2 = 0.5 * (1-(1-2*sigma_2)**w)
    zeta_1 = tau + eta_1 - 2 * tau * eta_1
    zeta_2 = tau + eta_2 - 2 * tau * eta_2
    a_1 = n * zeta_1 - xi
    b_1 = n * zeta_1 + xi
    a_2 = n * zeta_2 - xi
    b_2 = n * zeta_2 + xi
    print(a_1,b_1,a_2,b_2)
    while True:
        counter = counter + 1
#         if counter > 5000:
#             print ('Run signing too many times')
        
        e1 = gen_binom_vec(n, tau,field)
        e2 = gen_binom_vec(n, tau,field)
        y1 = e1 * B.transpose()
        y2 = e1 * B.transpose() + e2 * A.transpose()
        c1 = hash_function_1(l, y1, msg,field)
        r  = random_word(k - l, field)
        err = random_error(n, 32, field)
        t = vector(chain(r,c1)) * G_pub + err
        c2 = hash_function_2(l,w,t, y2, msg,field)
        z1 = e1 + c2 * X_i1.transpose()
        z2 = e2 + c2 * X_i2.transpose()
        wt_z1 = get_weight(z1)
        wt_z2 = get_weight(z2)
        print(wt_z1,wt_z2)
 
        
        # compute rejection rate
        rate_dict1, M1 = rejection_sampling_rate(n, get_weight(c2 * X_i1.transpose()), zeta_1, xi)
        rate_dict2, M2 = rejection_sampling_rate(n, get_weight(c2 * X_i2.transpose()), zeta_2, xi)
        

        if wt_z1 < int(a_1) or wt_z1 > int(b_1) or wt_z2 < int(a_2) or wt_z2 > int(b_2):
            continue

        if random.random() < rate_dict1[wt_z1]/M1 and random.random() < rate_dict2[wt_z2]/M2:
            print(counter)
            return (t, y2, z1, z2, msg)


In [22]:
n = 2048
k = 1696
l = 170
sigma_1 = 0.009
sigma_2 = 0.013
w = 17
xi = 20
r = 1696
m = 2048
tau = 0.09


# U = pk[0]
# B = pk[1]
# A = pk[2]
# G_pub = pk[3]
eta_1 = 0.5 * (1-(1-2*sigma_1)**w)
eta_2 = 0.5 * (1-(1-2*sigma_2)**w)
zeta_1 = tau + eta_1 - 2 * tau * eta_1
zeta_2 = tau + eta_2 - 2 * tau * eta_2
a_1 = n * zeta_1 - xi
b_1 = n * zeta_1 + xi
a_2 = n * zeta_2 - xi
b_2 = n * zeta_2 + xi

In [13]:
def gen_binom_vec(n, tau, field):
    v= [1 if random.random()<tau else 0 for _ in range(n)]
    v = vector(field,v)
    return v

In [24]:
msg = 'INK'
sig = sign(msg,n,k,l,w,sigma_1,sigma_2,tau,xi,pk,gsk,GF(2))

387.3920761746913 427.3920761746913 467.4427395015503 507.4427395015503
422 486
415 482
430 477
404 460
409 469
425 473
421 468
417 485
433 470
399 440
451 494
419 468
432 477
415 459
449 473
421 480
427 464
424 481
432 477
447 481
433 468
435 464
446 451
414 468
432 467
430 489
418 463
442 472
429 482
412 461
437 462
436 493
436 483
420 479
417 482
400 479
431 454
433 470
414 451
436 468
421 451
430 478
437 467
411 475
459 464
414 481
429 465
445 459
427 458
448 493
429 484
438 488
403 520
433 468
420 472
419 462
403 487
445 486
431 465
439 478
386 471
431 487
403 462
451 487
407 452
407 481
389 468
67


In [ ]:
show(sig[0])

In [ ]:
# print(getsize(A.str()))
# print(getsize(A.list()))

In [ ]:
print(get_weight(sig[2]))

In [ ]:
# print(len(vecmat))

In [ ]:
key_id = '0'
data_path = './data/'
key_suffix = '80' + '_' + key_id


filename = data_path + 'sig' + key_suffix
print('secret key saved in file ' + filename + '.sobj')
save(sig, filename)


In [28]:
def verify(sig,pk,n,k,l,w,sigma_1,sigma_2,tau,xi,field):
    t = sig[0]
    y2 = sig[1]
    z1 = sig[2]
    z2 = sig[3]
    msg = sig[4]
    U = pk[0]
    B = pk[1]
    A = pk[2]
    eta_1 = 0.5 * (1-(1-2*sigma_1)**w)
    eta_2 = 0.5 * (1-(1-2*sigma_2)**w)
    zeta_1 = tau + eta_1 - 2 * tau * eta_1
    zeta_2 = tau + eta_2 - 2 * tau * eta_2
    a_1 = n * zeta_1 - xi
    b_1 = n * zeta_1 + xi
    a_2 = n * zeta_2 - xi
    b_2 = n * zeta_2 + xi
    
#     y2 = vector(field,y2)
    c2 = hash_function_2(l,w,t, y2, msg,field)
    y2_prime = z1 * B.transpose() + z2 * A.transpose() - c2 * U.transpose()
#     y2_prime = vector(field,y2_prime)
    c2_prime = hash_function_2(l,w,t, y2_prime, msg,field)
    
    wt_z1 = get_weight(z1)
    wt_z2 = get_weight(z2)
#     print(y2_prime == y2)
#     print(c2_prime != c2)
    if wt_z1 < int(a_1) or wt_z1 > int(b_1) or wt_z2 < int(a_2) or wt_z2 > int(b_2) or c2_prime != c2:
        return 0
    
    return 1
    

In [29]:
print(verify(sig,pk,n,k,l,w,sigma_1,sigma_2,tau,xi,GF(2)))

1


In [30]:
N = 64

In [31]:
user_i = randint(0,N-1)

In [32]:
user_i

18

In [33]:
key_id = '0'
data_path = './data/'
key_suffix = '80' + '_' + key_id


gpklst = load(data_path + 'gpklst'+ key_suffix)
gsklst = load(data_path + 'gsklst'+ key_suffix)

In [34]:
gpk_i = gpklst[user_i]
gsk_i = gsklst[user_i]

In [36]:
msg = 'INK'
sig_i = sign(msg,n,k,l,w,sigma_1,sigma_2,tau,xi,pk,gsk_i,GF(2))

387.3920761746913 427.3920761746913 467.4427395015503 507.4427395015503
439 483
425 477
418 478
419 494
429 490
449 494
446 507
425 488
414 449
424 489
407 461
426 466
406 454
450 459
423 476
454 495
443 484
416 453
416 480
452 492
425 478
420 479
403 472
411 450
423 460
439 487
439 471
463 501
438 466
467 492
405 443
421 479
407 488
411 443
435 484
458 459
402 463
436 480
418 468
406 471
396 464
432 481
402 455
444 483
391 433
420 483
411 475
443 462
447 490
438 475
436 466
442 463
411 494
464 472
441 488
431 498
419 470
395 456
453 525
424 475
440 496
441 502
394 456
481 489
419 468
417 451
415 482
429 464
422 452
425 484
399 470
438 519
445 488
422 483
409 452
403 453
426 453
392 436
420 489
438 481
407 457
421 479
435 458
442 477
411 450
419 455
443 500
424 453
418 467
428 473
435 485
410 487
439 470
421 479
468 508
430 487
403 463
409 462
443 484
415 472
388 466
437 462
417 475
450 490
400 463
450 495
463 485
407 460
423 470
412 444
453 476
404 450
427 467
421 503
439 456
449 493


418 467
399 489
432 470
422 488
443 476
428 462
425 477
398 456
431 491
443 484
442 472
401 480
432 461
427 448
421 474
449 491
449 474
433 460
423 446
440 483
413 473
442 476
409 452
407 449
416 491
434 479
419 484
445 463
406 452
398 488
430 458
449 475
405 457
431 456
419 452
434 504
425 491
400 442
420 450
409 437
399 453
441 465
404 457
451 500
419 461
442 493
419 480
404 440
383 461
418 499
452 479
419 465
424 458
404 456
452 478
421 462
419 475
432 477
421 464
443 489
418 449
430 505
428 470
425 495
456 494
369 458
399 497
392 480
400 463
425 480
435 465
415 459
438 479
398 429
405 460
415 433
417 460
422 495
430 475
394 473
411 468
396 430
421 469
447 492
414 449
432 475
419 449
442 453
427 478
407 442
425 470
418 486
409 452
434 459
414 487
414 454
411 502
435 482
397 458
419 472
428 478
429 472
396 470
421 447
429 459
439 452
404 477
415 480
441 479
439 484
433 476
423 453
414 463
413 486
413 475
453 470
459 496
422 461
416 473
415 472
423 484
403 479
414 487
407 471
438 489


In [38]:
print(verify(sig_i,pk,n,k,l,w,sigma_1,sigma_2,tau,xi,GF(2)))

1


In [39]:
key_id = '0'
data_path = './data/'
key_suffix = '80' + '_' + key_id
gmsk = load(data_path + 'sk' + key_suffix)
# pk = load(data_path + 'pk')

In [40]:
lib_path = './'
load(lib_path + 'common.sage')
load(lib_path + 'goppacode.sage')
# load(lib_path + 'JoinImpl.sage')
# load(lib_path + 'KeyPrepare.sage')

In [41]:
S = gmsk[5]
P = gmsk[6]
b = gmsk[7]
g = gmsk[8]

G_pub = pk[3]
G_Goppa = S.inverse() * G_pub * P.transpose()

In [48]:
r = 1696
m = 2048
n = 2048
t = 32
#     show(U)
#     show(B)
#     show(A)
#     show(g1)

F = b.parent()
PR = g.parent()
Y = PR.gen()
K_ = GF(2)
L = [F(b**i) for i in range(2048)]
SyndromeCalculator = matrix(PR, 1, len(L));
for i in range(len(L)):
    SyndromeCalculator[0, i] = (Y - L[i]).inverse_mod(g);




In [58]:
def openSig(sig,pk,gpklst,g,X,SyndromeCalculator,L,G_Goppa,S,P,l,w,N,field):  
    t = sig[0]
    y2 = sig[1]
    z1 = sig[2]
    z2 = sig[3]
    msg = sig[4]
    B = pk[1]
    
    word = t * P.transpose()
    word_mat = matrix(K_,word)
#         yd = decodePatterson(word_mat,g1,X,SyndromeCalculator,L)
    word_prime = decodePatterson(word_mat,g,X,SyndromeCalculator,L)
    word_prime = vector(K_,word_prime.list())
    m = G_Goppa.solve_left(word_prime)
    m = m * S.inverse()
    
    c1 = m[1696-170:]
    c1 = vector(field,c1)
    c2 = hash_function_2(l,w,t, y2, msg,field)
    
    for i in range(N):
        G_i = gpklst[i]
        if c1 == hash_function_1(l,z1*B.transpose()-c2*G_i.transpose(), msg,field):
            return i
    
    return -1
    
    
    

In [60]:
N = 2 ** 6
openSig(sig_i,pk,gpklst,g,X,SyndromeCalculator,L,G_Goppa,S,P,l,w,N,GF(2))

18